In [ ]:
! pip install  hugchat  frontend transformers  pypdf2 chromadb  langchain pypdf sentence_transformers

# 1. Configure our variables
- ie Authentication essentials.
- it's from a temporary dummy email registered a hugging face account. Don't mind about it.

In [21]:
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ["EMAIL"] = userdata.get("EMAIL")
os.environ["PASS"] = userdata.get("PASS")

### Let's read the data from a pdf.
- Then create a vector database, here I used chromadb.
- Then created embeddings using hugging face because it's free and open source.
- Then used `test.pdf` as a sample to read the data (text) from, in this case I used a past paper in stocastic processes unit.

In [76]:
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.document_loaders import TextLoader
from langchain.document_loaders  import  PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.llms.base import LLM
from langchain.chains import RetrievalQA
import os
import warnings
import random
import string

document  = PyPDFLoader("/content/Mathematical_Modelling_in_Problem_Solving.pdf")
pages  = document.load()
pages[4]

Document(page_content='National Conference on Mathematics Education (NaCoME)\nIOP Conf. Series: Journal of Physics: Conf. Series 1480 (2020) 012001IOP Publishing\ndoi:10.1088/1742-6596/1480/1/012001\n4 \n \n \n \n \n \n \nFigure 2. Cartesian coordinate system . \n \nExample 5 . A 5 meters -height wall has a cube with 1 meter side length at the down -side of the wall. \nPlease determine the minimum length of a ladder that is laid on the wall and its’ end reaching the top \nof the wall.  \n4. Mathematical Model  \nReal world problems cannot be solved with mathematics before the problem is translated into \nmathematical language in the form of mathematical models. Therefore, the process of turning real \nworld problems into mathematical probl ems is called mathematical modelling. This is a process of \nmathematizing real -world problems and hence is also called a mathematical process [6, 7 ]. The steps \nof the mathematical modelling process for solving real -world  problems are as follow

In [77]:
warnings.filterwarnings("ignore")
repo_id = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceHubEmbeddings(
            repo_id=repo_id,
            task="feature-extraction"

        )
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, length_function = len,)
texts = text_splitter.split_documents(pages)
#random_str = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
#text = page.extract_text()

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


In [78]:
try:
  del db
except :
  pass

In [79]:
db  = Chroma.from_documents(texts, embeddings)

-  creating a Custom LLM from hugchat library. hugchat is unofficial hugging face API.
> I redesigned it to create a custom large language models inform of `langchains` format.

In [80]:
#The script has been made by wambugu kinyua.
#access the huggingchat api
#its a custom wrapper for a langchain llm
import time
from typing import Any, List, Mapping, Optional
from hugchat import hugchat
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM




class AiAmaizing_llm(LLM):

    """HuggingChat LLM wrapper."""

    chatbot : Optional[hugchat.ChatBot] = None

    email : Optional[str] = None
    psw : Optional[str] = None
    web_search: Optional[bool]= False
    temperature: Optional[float] = 0.2
    top_p: Optional[float] = 0.65
    repetition_penalty: Optional[float] = 1.2
    top_k: Optional[int]= 50
    truncate: Optional[int] = 1000
    watermark: Optional[bool] = False
    max_new_tokens: Optional[int] = 4096
    stop: Optional[list] = ["</s>"]
    return_full_text: Optional[bool] = False,
  #  stream: Optional[bool] = False,
    _stream_yield_all: Optional[bool] = False
    use_cache: Optional[bool] = False
    is_retry: Optional[bool] = False
    retry_count: Optional[int] = 5
    chatbot : Optional[hugchat.ChatBot] = None
  #  conversation: Optional[conversation] = None
    cookie_path : Optional[str] = None



    @property
    def _llm_type(self) -> str:
        return "Custom llm for llama2 HuggingChat api. Made by wambugu kinyua 🤠🥳"


    def create_chatbot(self) -> None:
        if not any([self.email, self.psw, self.cookie_path]):
            raise ValueError("email, psw, or cookie_path is required.")

        try:
            if self.email and self.psw:

                from hugchat.login import Login

                sign = Login(self.email, self.psw)
                cookies = sign.login()

            else:
                cookies = self.cookie_path and hugchat.ChatBot(cookie_path=self.cookie_path)

            self.chatbot = cookies.get_dict() and hugchat.ChatBot(cookies=cookies.get_dict())
        except Exception as e:
            raise ValueError("Login failed. Please check your email and password " + str(e))




    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop:
            raise ValueError("stop kwargs are not permitted.")

        self.create_chatbot() if not self.chatbot else None

        try:
#You can change the model
# index 0 for llama2 70b, 1 fo codellama, 2 for falcon 180b , 3 for Mistral 7b Ai
            self.chatbot.switch_llm(3) #falcon 180b
            self.chatbot.new_conversation(switch_to=True, system_prompt = "You  are  Ai-Squad-Amaizing Ai group, You are a  great helper")

            resp = self.chatbot.query(
                prompt, temperature=self.temperature,
        top_p= self.top_p,
        repetition_penalty = self.repetition_penalty,
        top_k = self.top_k,
        truncate= self.truncate,
        watermark= self.watermark,
        max_new_tokens = self.max_new_tokens,
        stop = self.stop,
        return_full_text = self.return_full_text,
       # stream = self.stream,
        _stream_yield_all = self._stream_yield_all,
        use_cache = self.use_cache,
        is_retry = self.is_retry,
        retry_count = self.retry_count,

        )
            return resp['text']

        except Exception as e:
            raise ValueError("ChatBot failed, please check your parameters. " + str(e))

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        params = {"web_search" :self.web_search,
"temperature": self.temperature,
"top_p": self.top_p,"repetition_penalty" : self.repetition_penalty,
"top_k" : self.top_k,"truncate" :self.truncate,"watermark" : self.watermark,"max_new_tokens" : self.max_new_tokens,
"stop" : self.stop,"return_full_text" : self.return_full_text,"_stream_yield_all" : self._stream_yield_all,
"use_cache" : self.use_cache,"is_retry"  : self.is_retry, "retry_count" : self.retry_count }

        return params

* let's test if the LLM is working.

In [81]:
llm = AiAmaizing_llm(email= os.environ["EMAIL"],psw = os.environ["PASS"])
llm("Who are  you?")

_getitem_
done


' I am an AI language model from the Ai Squad team. My purpose is to assist users with their questions and provide helpful information in various languages.'

- Let's query from our pdf.

In [86]:
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
#qa("what is data quality")
res = qa({"query": "in a detailed maths  working  work out  Example  4. A5  meter ... from  document"})#ask your question from your pdf
print(res["result"])

_getitem_
done
 To solve the problem of determining the maximum height of a ladder's back stuck on a wall, we can use the concept of trigonometry. We know that the angle between the ladder and the ground is equal to 90 degrees since the ladder is perpendicular to the ground. Let's denote this angle by "θ".

We also know that the ladder makes contact with the wall at two points, one at the bottom and one at the top. Let's call these points A and B respectively. We want to find the distance between point A and point C, which represents the maximum height of the ladder's back stuck on the wall.

Using the Pythagorean theorem, we can relate the distances AB and AC as follows:

AB^2 = AC^2 + BC^2

where BC is the horizontal distance between points B and C. Since the ladder is perpendicular to the wall, the vertical component of the ladder's height is equal to the difference between the heights of points A and B. Let's call this difference "h". Then, we have:

BC = h \* tan(θ)

Substituting 

## 2. The Ocr part
- Remember to change the filename according to your photos file name after uploading it.
- This is also a custom API that is free for OCR ie optical character Recognition😁.
- you can request 500 times a day under a single IP address.

In [ ]:
import requests
import os
import json

os.environ["API_KEY"] = "K89635879588957"

def ocr_space_file(filename, overlay=False, language='eng'):


    payload = {'isOverlayRequired': overlay,
               'apikey': os.environ["API_KEY"],
               'language': "eng",
               }
    with open(filename, 'rb') as f:
        response = requests.post('https://api.ocr.space/parse/image',
                          files={filename: f},
                          data=payload,
                          )
    return response

print("Processesing.......")

#text = ocr_space_file(filename='test1.jpg', language='eng')
results = ocr_space_file(filename='test3.jpg', language='eng').json()
#results = text["ParsedResults"][0]["ParsedText"]
res1 = results['ParsedResults'][0]['ParsedText']
#[0]["ParsedText"])
print(res1)

Processesing.......
3:05:50...
Queuing Mod...
LTE .111
4G 102
.lll
7.3 Characteristics of Queuing System
In designing a good queuing system, it is necessary to have a good information about the
model. The characteristics/features listed below would provide sufficient information.
i).
ii).
iii).
l).
11).
111).
IV).
Input (Arrival Pattern),
The service mechanism,
The queue discipline,
The number of service channels.
The input describe the manner in which the units arrive and join the system. The
system may have a limited or unlimited capacity for holding units. A unit may arrive
either singly or in a batch.
The arrival time for any customer is unpredictable. Therefore, the arrival time and
the number of customers arriving at any specified time intervals are usually random
variables. A Poisson distribution of arrivals correspond to arrivals at random. In
Poisson distribution, successive customers arrive after intervals which independently
are and exponentially distributed. The Poisson dis

## Testing the text from the image above.

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts_ = text_splitter.create_documents([res1])
db = Chroma.from_documents(texts_, embeddings)
retriever2 = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever2)
#qa("what is data quality")
res = qa({"query": "Examples?"})#ask your question from your pdf
print(res["result"])

_getitem_
done
 Sure! Here are some examples of queuing models:

1. M/M/N Model: This model represents a manufacturing process where multiple machines work together to produce goods. Customers represent jobs that need to be processed, while servers represent machines that perform tasks such as cutting, painting, or assembly. The arrival rate represents the rate at which new jobs come in, while the service rate represents the speed at which each machine processes its job. The number of machines (servers) determines how many jobs can be processed simultaneously.

2. G/G/N Model: This model represents a transportation network where vehicles travel along routes to pick up and deliver packages. Customers represent packages that need to be transported, while servers represent vehicles that move along the routes. The arrival rate represents the rate at which new packages arrive, while the service rate represents the speed at which each vehicle travels along the route. The number of vehicles (